# Multiclass (3 classes)

Try using 3 classes
1. 1-3
2. 4-8
3. 9-

* 1レースにつき100円で単勝を買う。
* 1レース中にデータが残っている馬数が出走頭数の半分以下の場合は買わない。

In [1]:
import tempfile
import warnings

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
import shap
import tqdm
import trueskill
from hyperopt import STATUS_OK, SparkTrials, Trials, fmin, hp, tpe
from hyperopt.pyll.base import scope
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
)
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler

from JapanHorseRaceAnalytics.utilities.base import (
    get_random_seed,
    get_spark_session,
    read_hive_table,
)
from JapanHorseRaceAnalytics.utilities.metrics import (
    calculate_binary_classifier_statistics,
    calculate_payout_rate,
    kelly_criterion,
)
from JapanHorseRaceAnalytics.utilities.plot import (
    plot_confusion_matrix,
    plot_correlation_matrix,
    plot_feature_importances,
    plot_shap_interaction_values,
)
from JapanHorseRaceAnalytics.utilities.structured_logger import logger

japanize_matplotlib.japanize()

In [2]:
spark = get_spark_session()

24/03/19 06:55:50 WARN Utils: Your hostname, Hanks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.40.105 instead (on interface en0)
24/03/19 06:55:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/03/19 06:55:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = read_hive_table(
    table_name="features_20240304_v1",
    schema="jhra_curated",
    spark_session=spark,
    use_cache=False,
    parse_dates=["meta_発走日時"],
)

rows_before = data.shape[0]
logger.info(f"Original data length: {rows_before}")

# Drop from data where cat_トラック種別 == "障害"
# Keep only horses that have 3 races
# Keep only data from 2000 onwards
data = data[
    # (data["cat_トラック種別"] != "障害")
    (~data["meta_着順"].isna())
    # & (data["meta_異常区分"] == "0")
    # & (data["num_1走前着順"].notnull())
    # & (data["num_2走前着順"].notnull())
    # & (data["num_3走前着順"].notnull())
    # & (data["meta_発走日時"] >= "2000-01-01")
]

rows_after = data.shape[0]
logger.info(
    f"Data length after filtering: {rows_after} (dropped {rows_before - rows_after} rows, {100 * (rows_before - rows_after) / rows_before:.2f}%)"
)

data.reset_index(drop=True, inplace=True)
data.head()

{"event": "Read from hive jhra_curated.features_20240304_v1", "level": "info", "timestamp": "2024-03-18T21:55:51.076623Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}
24/03/19 06:55:51 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/03/19 06:55:51 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/03/19 06:55:52 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
{"event": "Write to parquet /Users/hankehly/Projects/JapanHorseRaceAnalytics/data/sql_tables/features_20240304_v1.snappy.parquet", "level": "info", "timestamp": "2024-03-18T21:55:53.274384Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}
24/03/19 06:55:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
{"event": "Read from parquet /Users/hankehly/Projects/JapanHorseRaceAnalytics/data/sql_tables/features_2024030

,meta_単勝払戻金,meta_複勝払戻金,meta_レースキー,meta_馬番,meta_血統登録番号,meta_発走日時,meta_単勝的中,meta_単勝オッズ,meta_複勝的中,meta_複勝オッズ,...,cat_6走前休養理由分類コード,num_6走前3着タイム差,cat_トラック種別,num_距離,num_過去3走重み付き着順成績,num_入厩何日前逆数,cat_堅実な馬,cat_過去3走中1走訳あり凡走,cat_過去3走中2走好走,cat_過去3走繋がりあり
0,0,0,09025206,11,00100027,2002-12-01 12:45:00+09:00,0,38.6,0,4.4,...,None,NaN,芝,1600,NaN,1.0,False,False,False,False
1,0,0,08032303,04,00100027,2003-02-08 10:55:00+09:00,0,39.5,0,11.8,...,None,NaN,ダート,1800,NaN,1.0,False,False,False,False
2,0,0,08032801,14,00100027,2003-02-23 10:00:00+09:00,0,31.0,0,3.7,...,None,NaN,ダート,1800,NaN,1.0,False,False,False,False
3,0,0,09031403,07,00100027,2003-03-09 10:55:00+09:00,0,17.2,0,2.3,...,None,NaN,ダート,1800,0.173651,1.0,False,False,False,False
4,0,0,09031701,10,00100027,2003-03-22 10:05:00+09:00,0,20.8,0,2.7,...,None,NaN,ダート,1800,0.177772,1.0,False,False,False,False


In [4]:
# Assuming df is your DataFrame
df_sorted = data.sort_values(by=["meta_発走日時", "meta_レースキー"])

# Initialize the TrueSkill environment
env = trueskill.TrueSkill(draw_probability=0)  # No draws in horse racing

# Initialize ratings for all horses
horse_ratings = {
    horse_id: env.create_rating()
    for horse_id in df_sorted["meta_血統登録番号"].unique()
}

# Placeholder for ratings at each point in time
df_sorted["rating_post_race"] = pd.NA

# Iterate through races in chronological order
for (race_datetime, race_id), race_data in tqdm.tqdm(df_sorted.groupby(["meta_発走日時", "meta_レースキー"])):
    race_results = race_data.sort_values("meta_着順")
    horse_ids = race_results["meta_血統登録番号"].tolist()
    horse_groups = [[horse_ratings[horse_id]] for horse_id in horse_ids]
    ranks = list(range(len(horse_groups)))  # Lower rank number means a better position

    # Update ratings based on the race outcome
    updated_ratings = env.rate(horse_groups, ranks=ranks)

    # Directly update the DataFrame with the new ratings
    for index, (horse_id, new_rating_group) in zip(race_results.index, zip(horse_ids, updated_ratings)):
        horse_ratings[horse_id] = new_rating_group[0]  # Update with new rating
        df_sorted.at[index, "rating_post_race"] = horse_ratings[horse_id].mu  # Directly assign the rating

100%|██████████| 86237/86237 [02:41<00:00, 535.23it/s]


In [8]:
# Lag the rating_post_race by one row per horse (meta_血統登録番号) chronologically (meta_発走日時)
df_sorted["rating_pre_race"] = (
    df_sorted.sort_values("meta_発走日時")
    .groupby("meta_血統登録番号")["rating_post_race"]
    .shift(1)
    .fillna(25.0)
)

# Step 1: Calculate the total rating sum for each race
race_rating_sum = df_sorted.groupby("meta_レースキー")["rating_pre_race"].transform("sum")

# Step 2: Calculate the count of horses in each race
race_horse_count = df_sorted.groupby("meta_レースキー")["rating_pre_race"].transform("count")

# Step 3: Calculate the mean competitor rating for each horse
df_sorted["mean_competitor_rating_pre_race"] = (race_rating_sum - df_sorted["rating_pre_race"]) / (race_horse_count - 1)

# Step 4: Get the diff between the horse's rating and the mean competitor rating
df_sorted["mean_competitor_rating_pre_race_diff"] = df_sorted["rating_pre_race"] - df_sorted["mean_competitor_rating_pre_race"]

In [9]:
df_sorted["num_1走前標準化着順"] = (df_sorted["num_1走前着順"] - 1) / (df_sorted["num_1走前頭数"] - 1)
df_sorted["num_2走前標準化着順"] = (df_sorted["num_2走前着順"] - 1) / (df_sorted["num_2走前頭数"] - 1)
df_sorted["num_3走前標準化着順"] = (df_sorted["num_3走前着順"] - 1) / (df_sorted["num_3走前頭数"] - 1)

# We want our factor (num_1走前経過日数) to start from 0
# so subtract the minimum value for "days since last race" across all horses
# Add a small number to avoid division by zero
df_sorted["num_1走前重み"] = 1 / (df_sorted["num_1走前経過日数"] - df_sorted["num_1走前経過日数"].min() + 1e-6)
df_sorted["num_2走前重み"] = 1 / (df_sorted["num_2走前経過日数"] - df_sorted["num_1走前経過日数"].min() + 1e-6)
df_sorted["num_3走前重み"] = 1 / (df_sorted["num_3走前経過日数"] - df_sorted["num_1走前経過日数"].min() + 1e-6)

# Calculate weighted average of the feature
df_sorted["num_過去3走重み付き標準化着順"] = (
    (df_sorted["num_1走前標準化着順"] * df_sorted["num_1走前重み"])
    + (df_sorted["num_2走前標準化着順"] * df_sorted["num_2走前重み"])
    + (df_sorted["num_3走前標準化着順"] * df_sorted["num_3走前重み"])
) / (df_sorted["num_1走前重み"] + df_sorted["num_2走前重み"] + df_sorted["num_3走前重み"])

# 2. Weighted average time difference between the horse and the 3 horses behind it
df_sorted["num_1走前後続馬平均タイム差"] = (
    df_sorted[["num_1走前後続馬1タイム差", "num_1走前後続馬2タイム差", "num_1走前後続馬3タイム差"]]
    .mean(axis=1)
    .fillna(0)
)
df_sorted["num_2走前後続馬平均タイム差"] = (
    df_sorted[["num_2走前後続馬1タイム差", "num_2走前後続馬2タイム差", "num_2走前後続馬3タイム差"]]
    .mean(axis=1)
    .fillna(0)
)
df_sorted["num_3走前後続馬平均タイム差"] = (
    df_sorted[["num_3走前後続馬1タイム差", "num_3走前後続馬2タイム差", "num_3走前後続馬3タイム差"]]
    .mean(axis=1)
    .fillna(0)
)
df_sorted["num_過去3走重み付き後続馬平均タイム差"] = (
    (df_sorted["num_1走前後続馬平均タイム差"] * df_sorted["num_1走前重み"])
    + (df_sorted["num_2走前後続馬平均タイム差"] * df_sorted["num_2走前重み"])
    + (df_sorted["num_3走前後続馬平均タイム差"] * df_sorted["num_3走前重み"])
) / (df_sorted["num_1走前重み"] + df_sorted["num_2走前重み"] + df_sorted["num_3走前重み"])

In [42]:
df_final = df_sorted[
    (df_sorted["cat_トラック種別"] != "障害")
    & (df_sorted["num_1走前着順"].notnull())
    & (df_sorted["num_2走前着順"].notnull())
    & (df_sorted["num_3走前着順"].notnull())
]

# Remove rows where number of records for meta_レースキー is less than num_頭数 // 2
df_final = df_final[
    df_final.groupby("meta_レースキー")["meta_レースキー"].transform("count")
    > df_final["num_頭数"] // 2
]

In [43]:
def make_target(x):
    """
    https://qiita.com/Mshimia/items/6c54d82b3792925b8199#%E4%BA%88%E6%B8%AC%E3%83%A2%E3%83%87%E3%83%AB
    """
    if x <= 3:
        return 0
    elif x <= 8:
        return 1
    else:
        return 2


gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=get_random_seed())

for train_idx, test_idx in gss.split(df_final, groups=df_final["meta_レースキー"]):
    train_df = df_final.iloc[train_idx]
    test_df = df_final.iloc[test_idx]

# train_df and test_df contain your split data, respecting group boundaries
X_train = train_df.copy()
y_train = X_train["meta_着順"].apply(make_target)

X_test = test_df.copy()
y_test = X_test["meta_着順"].apply(make_target)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

X_train: (610942, 124)
X_test: (152462, 124)
y_train: (610942,)
y_test: (152462,)


In [171]:
def create_objective_fn(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    mlflow_experiment_name: str,
):
    """
    payouts should have the same index as *_test and have the following columns:
    * 発走日時
    * odds
    * payout
    """

    def train(params):
        mlflow.set_experiment(experiment_name=mlflow_experiment_name)
        with mlflow.start_run():
            preprocessor = ColumnTransformer(
                transformers=[
                    (
                        "num",
                        StandardScaler(),
                        [
                            "num_複勝率",
                            "num_1走前経過日数",
                            "num_過去3走重み付き標準化着順",
                            "num_過去3走重み付き後続馬平均タイム差",
                            "mean_competitor_rating_pre_race",
                            "rating_pre_race",
                        ],
                    ),
                ],
                remainder="drop",
            )

            X_train_prep = preprocessor.fit_transform(X_train, y_train)
            X_test_prep = preprocessor.transform(X_test)

            # Apply SMOTE
            smote_params = {
                k.split("__")[1]: v
                for k, v in params.items()
                if k.startswith("smote__")
            }
            smote = SMOTE(**smote_params)
            X_train_resampled, y_train_resampled = smote.fit_resample(
                X_train_prep, y_train
            )

            train_set = lgb.Dataset(X_train_resampled, label=y_train_resampled)
            valid_set = lgb.Dataset(X_test_prep, label=y_test, reference=train_set)

            classifier_params = {
                k.split("__")[1]: v
                for k, v in params.items()
                if k.startswith("classifier__")
            }
            model = lgb.train(
                params=classifier_params,
                train_set=train_set,
                valid_sets=[valid_set],
            )

            # Predict the class for each data point
            y_pred = np.argmax(model.predict(X_test_prep), axis=1)

            # If you want the probabilities to calculate log loss
            y_pred_proba = model.predict(X_test_prep)

            metrics = {
                "loss": -precision_score(y_test, y_pred, average="weighted"),
                "accuracy": accuracy_score(y_test, y_pred),
                "precision": precision_score(y_test, y_pred, average="weighted"),
                "recall": recall_score(y_test, y_pred, average="weighted"),
                "f1": f1_score(y_test, y_pred, average="weighted"),
                "log_loss": log_loss(y_test, y_pred_proba),
            }

            mlflow.log_params(classifier_params)
            mlflow.log_metrics(metrics)

            # Suppress UserWarning messages from matplotlib
            warnings.filterwarnings("ignore", category=UserWarning)

            # Confusion Matrix
            fig, _ = plot_confusion_matrix(y_test, y_pred)
            with tempfile.NamedTemporaryFile(
                prefix="confusion_matrix_", suffix=".png"
            ) as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances Plot (Gain)
            fig, ax = plot_feature_importances(
                feature_names=preprocessor.get_feature_names_out(),
                feature_importances=model.feature_importance(importance_type="gain"),
                top_n=50,
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_gain_", suffix=".png"
            ) as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances Data (Gain)
            feature_importances = zip(
                preprocessor.get_feature_names_out(),
                model.feature_importance(importance_type="gain"),
            )
            feature_importances_df = (
                pd.DataFrame(feature_importances, columns=["feature", "importance"])
                .sort_values("importance", ascending=False)
                .reset_index(drop=True)
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_gain_", suffix=".csv"
            ) as f:
                feature_importances_df.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)

            # Feature Importances Plot (Split)
            fig, ax = plot_feature_importances(
                feature_names=preprocessor.get_feature_names_out(),
                feature_importances=model.feature_importance(importance_type="split"),
                top_n=50,
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_split_", suffix=".png"
            ) as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances Data (Split)
            feature_importances = zip(
                preprocessor.get_feature_names_out(),
                model.feature_importance(importance_type="split"),
            )
            feature_importances_df = (
                pd.DataFrame(feature_importances, columns=["feature", "importance"])
                .sort_values("importance", ascending=False)
                .reset_index(drop=True)
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_split_", suffix=".csv"
            ) as f:
                feature_importances_df.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)

            # SHAP values
            X_test_sample = X_test.sample(n=10_000, random_state=get_random_seed())
            X_test_sample_prep = preprocessor.transform(X_test_sample)
            explainer = shap.TreeExplainer(
                model=model,
                feature_names=preprocessor.get_feature_names_out(),
            )
            # 0 is the class index for True
            shap_values = explainer(X_test_sample_prep)[:, :, 0]
            shap_interaction_values = explainer.shap_interaction_values(
                X_test_sample_prep
            )

            # SHAP beeswarm plot
            shap.plots.beeswarm(shap_values, show=False)
            plt.tight_layout()
            with tempfile.NamedTemporaryFile(
                prefix="shap_beeswarm_", suffix=".png"
            ) as f:
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # SHAP interaction values heatmap
            fig, ax = plot_shap_interaction_values(
                shap_interaction_values, preprocessor.get_feature_names_out()
            )
            with tempfile.NamedTemporaryFile(
                prefix="shap_interactions_", suffix=".png"
            ) as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # SHAP bar plot
            shap.plots.bar(shap_values, show=False)
            plt.tight_layout()
            with tempfile.NamedTemporaryFile(prefix="shap_bar_", suffix=".png") as f:
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Correlation matrix
            fig, ax = plot_correlation_matrix(
                data=preprocessor.transform(X_test),
                columns=preprocessor.get_feature_names_out(),
            )
            with tempfile.NamedTemporaryFile(
                prefix="correlation_matrix_", suffix=".png"
            ) as f:
                fig.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)
            return {"status": STATUS_OK, "params": params, "model": model, **metrics}

    return train

In [172]:
space = {
    # Prefix SMOTE parameters with "smote__"
    "smote__random_state": get_random_seed(),
    # Prefix classifier parameters with "classifier__"
    "classifier__num_leaves": scope.int(hp.quniform("num_leaves", 20, 150, 1)),
    "classifier__max_depth": scope.int(hp.quniform("max_depth", 3, 10, 1)),
    "classifier__learning_rate": hp.loguniform("learning_rate", -5, 0),
    "classifier__n_estimators": scope.int(hp.quniform("n_estimators", 100, 1000, 1)),
    "classifier__min_child_samples": scope.int(
        hp.quniform("min_child_samples", 20, 500, 1)
    ),
    "classifier__subsample": hp.uniform("subsample", 0.5, 1),
    "classifier__colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "classifier__reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
    "classifier__reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
    "classifier__feature_fraction": hp.uniform("feature_fraction", 0.5, 1.0),
    "classifier__lambda_l1": hp.uniform("lambda_l1", 0, 5),
    "classifier__lambda_l2": hp.uniform("lambda_l2", 0, 5),
    "classifier__min_split_gain": hp.uniform("min_split_gain", 0, 1),
    "classifier__min_child_weight": hp.uniform("min_child_weight", 0.001, 10),
    "classifier__boosting_type": "gbdt",
    "classifier__objective": "multiclassova",
    "classifier__num_class": 3,
    "classifier__metric": "multi_error",
    "classifier__verbose": -1,
    "classifier__random_state": get_random_seed(),
}

In [173]:
experiment_name = "20240318_eda2_3class"

if mlflow.get_experiment_by_name(experiment_name) is None:
    mlflow.create_experiment(experiment_name)

fn = create_objective_fn(
    X_train,
    y_train,
    X_test,
    y_test,
    mlflow_experiment_name=experiment_name,
)

In [3]:
# trials = Trials()
# fmin(
#     fn=fn,
#     space=space,
#     algo=tpe.suggest,
#     max_evals=1,
#     trials=trials,
# )

trials = SparkTrials(parallelism=3, spark_session=spark)
best = fmin(fn=fn, space=space, algo=tpe.suggest, max_evals=30, trials=trials)
best

{'colsample_bytree': 0.6200128394071927,
 'feature_fraction': 0.6369370526744543,
 'lambda_l1': 2.974353146652003,
 'lambda_l2': 0.03017023287643028,
 'learning_rate': 0.04177858173712023,
 'max_depth': 5.0,
 'min_child_samples': 238.0,
 'min_child_weight': 7.497597144935071,
 'min_split_gain': 0.07812159082885184,
 'n_estimators': 972.0,
 'num_leaves': 143.0,
 'reg_alpha': 0.14861641212382748,
 'reg_lambda': 0.6538265648271644,
 'subsample': 0.7626081055867588}

In [44]:
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            [
                "num_複勝率",
                "num_1走前経過日数",
                "num_過去3走重み付き標準化着順",
                "num_過去3走重み付き後続馬平均タイム差",
                "mean_competitor_rating_pre_race",
                "mean_competitor_rating_pre_race_diff",
                "rating_pre_race",
            ],
        ),
    ],
    remainder="drop",
)

X_train_prep = preprocessor.fit_transform(X_train, y_train)
X_test_prep = preprocessor.transform(X_test)

smote = SMOTE(random_state=get_random_seed())
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_prep, y_train)

train_set = lgb.Dataset(X_train_resampled, label=y_train_resampled)
valid_set = lgb.Dataset(X_test_prep, label=y_test, reference=train_set)

classifier_params = {
    "colsample_bytree": 0.6200128394071927,
    "feature_fraction": 0.6369370526744543,
    "lambda_l1": 2.974353146652003,
    "lambda_l2": 0.03017023287643028,
    "learning_rate": 0.04177858173712023,
    "max_depth": 5,
    "min_child_samples": 238,
    "min_child_weight": 7.497597144935071,
    "min_split_gain": 0.07812159082885184,
    "n_estimators": 972,
    "num_leaves": 143,
    "reg_alpha": 0.14861641212382748,
    "reg_lambda": 0.6538265648271644,
    "subsample": 0.7626081055867588,
    "boosting_type": "gbdt",
    "objective": "multiclassova",
    "num_class": 3,
    "metric": "multi_error",
    "verbose": -1,
    "random_state": get_random_seed(),
}

model = lgb.train(
    params=classifier_params,
    train_set=train_set,
    valid_sets=[valid_set],
)

# Predict the class for each data point
y_pred = np.argmax(model.predict(X_test_prep), axis=1)

# If you want the probabilities to calculate log loss
y_pred_proba = model.predict(X_test_prep)

metrics = {
    "loss": -precision_score(y_test, y_pred, average="weighted"),
    "accuracy": accuracy_score(y_test, y_pred),
    "precision": precision_score(y_test, y_pred, average="weighted"),
    "recall": recall_score(y_test, y_pred, average="weighted"),
    "f1": f1_score(y_test, y_pred, average="weighted"),
    "log_loss": log_loss(y_test, y_pred_proba),
}

Found `n_estimators` in params. Will use it instead of argument
The y_pred values do not sum to one. Starting from 1.5 thiswill result in an error.


In [47]:
payouts = (
    X_test[["meta_レースキー", "meta_血統登録番号", "meta_着順", "meta_発走日時", "meta_単勝オッズ", "meta_複勝オッズ"]]
    .assign(meta_発走日時=lambda x: pd.to_datetime(x["meta_発走日時"]))
    .assign(y_test=y_test, y_pred=y_pred, y_pred_proba_0=y_pred_proba[:, 0])
)

bets = []
for i, (race_key, group) in enumerate(payouts.groupby("meta_レースキー")):
    top_horse = group.sort_values("y_pred_proba_0", ascending=False).iloc[0]
    if top_horse.y_pred == 0:
        bets.append([race_key, top_horse["meta_血統登録番号"], True])

payouts = payouts.merge(
    pd.DataFrame(bets, columns=["meta_レースキー", "meta_血統登録番号", "bet"]),
    on=["meta_レースキー", "meta_血統登録番号"],
    how="left",
).fillna({"bet": False})

In [48]:
(
    payouts
    .assign(hit_f=lambda x: x["bet"] & (x["meta_着順"] <= 3))
    .assign(miss_f=lambda x: x["bet"] & ~x["hit_f"])
    .assign(payout_f=lambda x: x["hit_f"] * (x["meta_複勝オッズ"] * 100 - 100) - x["miss_f"] * 100)

    .assign(hit_t=lambda x: x["bet"] & (x["meta_着順"] == 1))
    .assign(miss_t=lambda x: x["bet"] & ~x["hit_t"])
    .assign(payout_t=lambda x: x["hit_t"] * (x["meta_単勝オッズ"] * 100 - 100) - x["miss_t"] * 100)

    # .reset_index(drop=True)
    .sort_values(["meta_発走日時", "meta_レースキー"])
    # .dropna(subset=["meta_単勝オッズ", "meta_複勝オッズ"])
)[["payout_t", "payout_f"]].sum()

payout_t   -226170.0
payout_f   -283620.0
dtype: float64

In [ ]:
# Bankroll over time
# results = (
#     pd.concat(
#         [
#             payouts,
#             pd.Series(y_pred.astype(bool)).rename("pred"),
#             pd.Series(y_proba[:, 0]).rename("proba_true"),
#             y_test.astype(bool).reset_index(drop=True).rename("actual"),
#         ],
#         axis=1,
#     )
#     .set_index("発走日時")
#     .sort_index()
#     .dropna()
# )
# b = results["odds"] - 1
# p = results["proba_true"]
# q = 1 - p
# japanize_matplotlib.japanize()
# for confidence in [0.5, 0.65, 0.8]:
#     fig, ax = plt.subplots(figsize=(15, 5))
#     for multiplier in [0.1, 0.2, 0.3]:
#         results[f"kelly_{multiplier}"] = (
#             kelly_criterion(b, p, q).clip(lower=0) * multiplier
#         )
#         bankroll = 10_000
#         history = []
#         for i, row in results.iterrows():
#             # bet in 100 yen increments
#             bet_amount = (
#                 round(row[f"kelly_{multiplier}"] * bankroll / 100) * 100
#             )
#             bet = bet_amount > 0 and row["proba_true"] >= confidence
#             if bet and row["actual"] is True:
#                 bankroll += (row["odds"] - 1) * bet_amount
#             elif bet and row["actual"] is False:
#                 bankroll -= bet_amount
#             history.append(bankroll)
#         results["bankroll"] = history
#         results["bankroll"].plot(
#             ax=ax, label=f"Kelly Criterion x {multiplier}"
#         )
#     ax.set_title("Bankroll over time (Confidence: {confidence})")
#     ax.set_ylabel("Bankroll")
#     ax.set_xlabel("Date")
#     ax.legend()
#     plt.tight_layout()
#     plt.grid()
#     with tempfile.NamedTemporaryFile(
#         prefix=f"bets_{confidence}_confidence_", suffix=".png"
#     ) as f:
#         fig.savefig(f.name)
#         plt.close()
#         mlflow.log_artifact(f.name)